<a href="https://colab.research.google.com/github/Vinaypatil-Ev/vinEvPy-GoCoLab/blob/main/Tensorflow/TensorflowPrac18_customize_modedl_fit().ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

## Customize model.fit

In [ ]:
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            ypred = self(x, training=True)

            loss = self.compiled_loss(y, ypred, regularization_losses=self.losses)
        
        grad = tape.gradient(loss, self.trainable_variables)

        self.optimizer.apply_gradients(zip(grad, self.trainable_variables))

        self.compiled_metrics.update_state(y, ypred)

        return {m.name: m.result() for m in self.metrics}

In [ ]:
inputs = tf.keras.Input((100))
x = tf.keras.layers.Dense(64, activation="relu")(inputs)
outputs = tf.keras.layers.Dense(1, activation='softmax')(x)
model = CustomModel(inputs, outputs)
model.compile("adam", "mse", metrics=["mae"])

In [ ]:
x = tf.random.normal((1, 100))
y = tf.random.normal((1, 1))

In [ ]:
model.fit(x, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 0.8169 - mae: 0.9038
Epoch 2/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8169 - mae: 0.9038
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.8169 - mae: 0.9038
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 0.8169 - mae: 0.9038
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 0.8169 - mae: 0.9038
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: 0.8169 - mae: 0.9038
Epoch 7/10
1/1 [==============================] - 0s 4ms/step - loss: 0.8169 - mae: 0.9038
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 0.8169 - mae: 0.9038
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 0.8169 - mae: 0.9038
Epoch 10/10
1/1 [==============================] - 0s 3ms/step - loss: 0.8169 - mae: 0.9038


## Customize compile()

In [ ]:
msem = tf.keras.metrics.MeanAbsoluteError(name="mse")
lossm = tf.keras.metrics.Mean(name="loss")
class CustomCompile(tf.keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            ypred = self(x, training=True)
            loss = tf.keras.losses.mean_squared_error(y, ypred)
        
        grad = tape.gradient(loss, self.trainable_variables)

        self.optimizer.apply_gradients(zip(grad, self.trainable_variables))
        
        lossm.update_state(loss)

        msem.update_state(y, ypred)


        return {"loss": msem.result(), "mse": lossm.result()}

    @property
    def metrics(self):
        return [lossm, msem]

In [ ]:
inputs = tf.keras.Input((32, ))
outputs = tf.keras.layers.Dense(1, activation="relu")(inputs)
model = CustomCompile(inputs, outputs)
model.compile("adam")

In [ ]:
x = tf.random.uniform((100, 32))
y = tf.random.normal((100, 1))
model.fit(x, y, epochs=5)

Epoch 1/5
4/4 [==============================] - 0s 4ms/step - loss: 0.8616 - mse: 1.0883
Epoch 2/5
4/4 [==============================] - 0s 3ms/step - loss: 0.8613 - mse: 1.0874
Epoch 3/5
4/4 [==============================] - 0s 4ms/step - loss: 0.8609 - mse: 1.0868
Epoch 4/5
4/4 [==============================] - 0s 4ms/step - loss: 0.8607 - mse: 1.0863
Epoch 5/5
4/4 [==============================] - 0s 4ms/step - loss: 0.8604 - mse: 1.0858


# GAN

In [ ]:
descriminator = tf.keras.Sequential([
    tf.keras.Input((28, 28, 1)),
    tf.keras.layers.Conv2D(64, 3, strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(128, 3, strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(1)
])


In [ ]:
latent_dim = 128

generator = tf.keras.Sequential([
    tf.keras.Input(latent_dim),
    tf.keras.layers.Dense(7 *  7 * 128),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Reshape((7, 7, 128)),
    tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid")
])

In [ ]:
class GAN(tf.keras.Model):
    def __init__(self, descriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.descriminator = descriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_otimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
    
    def train_step(self, real_img):
        if isinstance(real_img, tuple):
            real_img = real_img[0]
        
        batch_size = tf.shape(real_img)[0]
        random_latent_vector = tf.random.normal(shape=(batch_size, self.latent_dim))
        generated_img = self.generator(random_latent_vector)
        combined_img = tf.concat([real_img, generated_img], axis=0)
        label = tf.concat([
                           tf.ones((batch_size, 1)),
                           tf.zeros((batch_size, 2))
                           ], axis=0)
        label = label + 0.5 * tf.random.uniform(tf.shape(label))

        with tf.GradientTape() as d_tape:
            d_pred = self.descriminator(combined_img)
            d_loss = self.loss_fn(label, d_pred)

        d_grad = d_tape.gradient(d_loss, self.descriminator.trainable_weights)
        self.d_optimizer.apply_gradient(zip(d_grad, self.descriminator.trainable_weights))

        random_latent_vector = tf.random.normal(shape=(batch_size, self.latent_dim))
        false_labels = tf.ones((batch_size, 1))

        with tf.GradientTape() as g_tape:
            g_pred = self.generator(self.descriminator(random_latent_vector))
            g_loss = self.loss_fn(false_labels, g_pred)
        
        g_grad = g_tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_otimizer.apply_gradients(zip(g_grad, self.trainable_weights))

        return {"d_loss": d_loss, "g_loss": g_loss}




In [ ]:
batch_size = 64
(x_trn, _), (x_tst, _) = tf.keras.datasets.mnist.load_data()
x = np.concatenate([x_trn, x_tst])
x = x.astype("float32") / 255.0
x = np.reshape(x, (-1, 28, 28, 1))
data = tf.data.Dataset.from_tensor_slices(x)
data = data.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
gan = GAN(descriminator, generator, latent_dim)
gan.compile(
    tf.keras.optimizers.Adam(learning_rate=0.0003),
    tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
)

In [ ]:
gan.fit(data.take(100), epochs=1)

In [ ]:
descriminator = tf.keras.Sequential([
    tf.keras.Input((28, 28, 1)),
    tf.keras.layers.Conv2D(64, 3, strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(128, 3, strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(1)
])

In [ ]:
latent_dim = 128

generator = tf.keras.Sequential([
    tf.keras.Input(shape=(latent_dim,)),
    tf.keras.layers.Dense(7 *  7 * 128),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Reshape((7, 7, 128)),
    tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
    tf.keras.layers.LeakyReLU(alpha=0.2),
    tf.keras.layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid")
])

In [ ]:
inputs = tf.keras.Input((28, 28, 1))
x = generator(inputs)
outputs = descriminator(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# tf.keras.utils.plot_model(descriminator, show_shapes=True)

In [ ]:
# tf.keras.utils.plot_model(generator, show_shapes=True)